In [ ]:
import pandas as pd

# V1
df = pd.read_csv('./data/Journals/FinancialJournal.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df.drop_duplicates(inplace=True)

df = df.groupby(['participantId', 'year', 'month', 'category'])['amount'].sum().reset_index()
df['date'] = (df['year'] * 100 + df['month']).astype(str)
df['date'] = pd.to_datetime(df['date'], format='%Y%m')

df = df.set_index('date').groupby(['participantId','category'])['amount'].resample('MS').asfreq().reset_index()

bussiness_area = ['Education', 'Food', 'Recreation', 'Shelter']
df = df.query('category in @bussiness_area').reset_index(drop=True)
df['amount'] = df['amount'].abs()
df.to_csv('./data/v1.csv', index=False)
# pd.pivot_table(df, index=['participantId', 'year', 'month'], columns=['category'], values=['amount'], aggfunc='sum')


In [9]:
jobs = pd.read_csv('data/Attributes/Jobs.csv')
employers = pd.read_csv('data/Attributes/Employers.csv')
resaurants = pd.read_csv('data/Attributes/Restaurants.csv')
pubs = pd.read_csv('data/Attributes/Pubs.csv')

In [22]:
# V2
import os
import pandas as pd

res = pd.DataFrame()
for f in os.listdir('data/Activity_Logs/'):
    if not f.endswith('csv'):
        continue
    else:
        df = pd.read_csv(f'data/Activity_Logs/{f}')
        df = df.query('currentMode == "AtRestaurant"').reset_index(drop=True)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['year'] = df['timestamp'].dt.year
        df['month'] = df['timestamp'].dt.month

        tmp = df.groupby(['participantId', 'year', 'month'])['timestamp'].count().reset_index(name='hour')
        res = pd.concat([res, tmp], ignore_index=True)

res = res.groupby(['participantId', 'year', 'month'])['hour'].sum().reset_index()  # this is popularity
res['hour'] = res['hour'] * 5 / 60
res['participantId'] = res['participantId'].astype(int)
res['date'] = pd.to_datetime((res['year'] * 100 + res['month']).astype(str), format='%Y%m')
res = res[['participantId', 'date', 'hour']].copy()

df = pd.read_csv('data/Journals/FinancialJournal.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.drop_duplicates(inplace=True)
df['timestamp'] = df['timestamp'].agg(lambda x: x.date)
df = df.query('category == "Wage"').groupby(['participantId', 'timestamp'], as_index=False)['amount'].sum()
df['year'] = pd.to_datetime(df['timestamp']).dt.year
df['month'] = pd.to_datetime(df['timestamp']).dt.month
res_sa = df.groupby(['participantId', 'year', 'month'], as_index=False)['amount'].mean()
res_sa['date'] = pd.to_datetime((res_sa['year'] * 100 + res_sa['month']).astype(str), format='%Y%m')
res_sa = res_sa[['participantId', 'date', 'amount']].copy()

res = res.merge(res_sa, on=['participantId', 'date'], how='left')
res.to_csv('data/v2.csv', index=False)


In [110]:
# V3
df = pd.read_csv('./data/Journals/FinancialJournal.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df.drop_duplicates(inplace=True)

expense_list = ['Shelter', 'Education', 'Food', 'Recreation']
expense_res = df.query('category in @expense_list').groupby(
    ['participantId', 'year', 'month'], as_index=False)['amount'].sum()
expense_res['amount'] = expense_res['amount'].abs()
wage_res = df.query('category == "Wage"').groupby(
    ['participantId', 'year', 'month'], as_index=False
)['amount'].sum()

res = wage_res.merge(expense_res, on=['participantId', 'year', 'month'], how='left')
res['engel_idx'] = res['amount_y'] / res['amount_x']
res['date'] = pd.to_datetime((res['year'] * 100 + res['month']).astype(str), format='%Y%m')
res.to_csv('data/v3.csv', index=False)

In [111]:
res

,participantId,year,month,amount_x,amount_y,engel_idx,date
0,0,2022,3,11931.947623,1210.049425,0.101413,2022-03-01
1,0,2022,4,8636.880059,1078.279044,0.124846,2022-04-01
2,0,2022,5,9048.160061,1240.618894,0.137113,2022-05-01
3,0,2022,6,9048.160061,1315.647934,0.145405,2022-06-01
4,0,2022,7,8636.880059,1932.750901,0.223779,2022-07-01
...,...,...,...,...,...,...,...
13326,1010,2023,1,2123.906450,1176.618262,0.553988,2023-01-01
13327,1010,2023,2,1930.824046,1669.137946,0.864469,2023-02-01
13328,1010,2023,3,2220.447652,1683.335827,0.758107,2023-03-01
13329,1010,2023,4,1930.824046,1200.646897,0.621831,2023-04-01


In [178]:
# V4
part_info = pd.read_csv('data/Attributes/Participants.csv')
part_info['mood_type'] = part_info['joviality'].agg(lambda x: 'happy' if x >= 0.5 else 'sad')
part_info = part_info[['participantId', 'educationLevel', 'mood_type', 'haveKids']].copy()
df = pd.read_csv('data/Journals/FinancialJournal.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df.drop_duplicates(inplace=True)
res = df.groupby(['participantId', 'year', 'month', 'category'], as_index=False)['amount'].sum()
res['amount'] = res['amount'].abs()
pres = pd.pivot_table(res, index=['participantId', 'year', 'month'], columns=['category'], values=['amount']).reset_index()
pres.columns = pres.columns.to_flat_index()
pres.columns = [''.join(col) for col in pres.columns]
pres.drop(columns=['amountRentAdjustment'], axis=1, inplace=True)
pres = pres.fillna(0)
pres['edu_ratio'] = pres['amountEducation'] / pres['amountWage']
pres['food_ratio'] = pres['amountFood'] / pres['amountWage']
pres['recreation_ratio'] = pres['amountRecreation'] / pres['amountWage']
pres['shelter_ratio'] = pres['amountShelter'] / pres['amountWage']
pres = pres[['participantId', 'year', 'month', 'edu_ratio', 'food_ratio', 'recreation_ratio', 'shelter_ratio']].copy()
pres = pres.merge(part_info, on=['participantId'], how='left')
pres.to_csv('data/v4.csv', index=False)

In [186]:
# V5
part_info = pd.read_csv('data/Attributes/Participants.csv')
part_info['mood_type'] = part_info['joviality'].agg(lambda x: 'happy' if x >= 0.5 else 'sad')
part_info = part_info[['participantId', 'educationLevel', 'mood_type', 'haveKids']].copy()
df = pd.read_csv('data/Journals/FinancialJournal.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df.drop_duplicates(inplace=True)
res = df.groupby(['participantId', 'year', 'month', 'category'], as_index=False)['amount'].sum()
res['amount'] = res['amount'].abs()
expense_list = ['Education', 'Food', 'Recreation', 'Shelter']  # Wage
exp_res = res.query('category in @expense_list').groupby(['participantId', 'year', 'month'], as_index=False)['amount'].sum()
w_res = res.query('category == "Wage"').groupby(['participantId', 'year', 'month'], as_index=False)['amount'].mean()

In [202]:
tmp = exp_res.merge(w_res, on=['participantId', 'year', 'month'], how='left')
tmp['saving_rate'] = 1 - (tmp['amount_x'] / tmp['amount_y'])
tmp = tmp[['participantId', 'year', 'month', 'saving_rate']].copy()
tmp = tmp.merge(part_info, how='left', on=['participantId'])
tmp['date'] = pd.to_datetime((tmp['year'] * 100 + tmp['month']).astype(str), format='%Y%m')
tmp.drop(columns=['year', 'month'], axis=1, inplace=True)
tmp.to_csv('data/v5.csv', index=False)